### Connect to Azure Machine Learning workspace

In [18]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
client = MLClient.from_config(credential)


Found the config file in: .\config.json


### Get Credentials from Key Vault

In [20]:
from azure.keyvault.secrets import SecretClient

keyvault_uri = "https://plato-default-key-vault.vault.azure.net/"
secret_client = SecretClient(vault_url=keyvault_uri, credential=credential)

tenant_id = secret_client.get_secret('azure-tenant-id').value
client_id = secret_client.get_secret('ml-client-id').value
client_secret = secret_client.get_secret('ml-client-secret').value

### Configure the Command

In [22]:
from azure.ai.ml import command, Input, Output, UserIdentityConfiguration
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes

identity = UserIdentityConfiguration()
registered_model_name = "plato-gpt-trained-model"

input_path = 'azureml://subscriptions/af9d95ee-d424-470c-8f1a-3540102fef9f/resourcegroups/plato/workspaces/plato-workspace/datastores/workspaceblobstore/paths/sam_harris_dataset/data.txt'
input_data_type = AssetTypes.URI_FILE
input_mode = InputOutputModes.RO_MOUNT

config_path = './config.json'
config_file_type = AssetTypes.URI_FILE
config_file_mode = InputOutputModes.RO_MOUNT

output_path = 'azureml://subscriptions/af9d95ee-d424-470c-8f1a-3540102fef9f/resourcegroups/plato/workspaces/plato-workspace/datastores/workspaceblobstore/paths/azureml/'
output_data_type = AssetTypes.URI_FOLDER
output_mode = InputOutputModes.RW_MOUNT

env = 'azureml:Plato-GPT-env:2'


# configure the command job
job = command(
    inputs=dict(
        data=Input(
            type=input_data_type,
            path=input_path,
            mode=input_mode
        ),
        config=Input(
            type=config_file_type,
            path=config_path,
            mode=config_file_mode
        ), 
        start_fresh = False
    ),
    outputs=dict(
        log_out=Output(
            type=output_data_type,
            path=output_path,
            output_mode=output_mode
        ),
    ),
    environment_variables= {
        'AZURE_TENANT_ID': tenant_id,
        'AZURE_CLIENT_ID': client_id,
        'AZURE_CLIENT_SECRET': client_secret,
    },
    code="./src",  # location of source code
    command="python train.py --data ${{inputs.data}} --config ${{inputs.config}} --start_fresh ${{inputs.start_fresh}}",
    environment=env,
    compute='nv12',
    experiment_name="Plato-GPT-2-char",
    display_name="Plato-GPT-tr-lr_low-resume-ver",
    identity=identity
)

In [23]:
client.create_or_update(job)

Uploading src (0.06 MBs): 100%|##########| 57735/57735 [00:00<00:00, 84235.95it/s]




Experiment,Name,Type,Status,Details Page
Plato-GPT-2-char,mango_shark_g3135vdw3v,command,Starting,Link to Azure Machine Learning studio
